# Agents in Langchain

## Agents

Autonomous systems that make decisions and take actions. They can solve math problems, perform searches on wikipedia or decide when to switch between LLM and tools for an specific task for instance.

Agents can *reason* and *act* (ReAct)

## Tools 

Functions used by agents to perform specific tasks (querying data, research reports, data analysis...)

## Graphs

LangGraph is another framework from the langchain family that that allows to create graphs (workflows) of agents and tools. In these graphs, tasks (nodes) are connected by edges (rules)


In [37]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
import math
import os


In [38]:
model = ChatOpenAI(openai_api_key = os.environ["OPENAI_API_KEY"], model='gpt-4o-mini')


def count_r_in_word(word: str) -> int:
    """Counts the number of occurrences of the letter 'r' in a given word."""
    return word.lower().count("r")

agent = create_react_agent(model, tools=[count_r_in_word])

query = "WHow many r's are in the word 'Terrarium'?"

response = agent.invoke({"messages": [("human", query)]})


In [ ]:
print(response['messages'][-1].content)

## Building custom functions 

Although there is already a set of pre-built and custom tools avaiable for database querying, web scrapping, image generation, etc. you can create your own tools.


In [ ]:
@tool 
def rectangle_area(input: str) -> int:
    """Calculates the area of a rectangle given its length and width."""
    sides = input.split(",")
    a = float(sides[0].strip())
    b = float(sides[1].strip())
    return a*b

tools=[rectangle_area]

query = "What is the area of a rectangle with sides 10, 20?"

agent = create_react_agent(model, tools=tools)

response = agent.invoke({"messages": [("human", query)]})
print(response['messages'][-1].content)


In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

message_history = response["messages"]
new_query = "What about one with sides 12 and 14?"

# Invoke the app with the full message history
response = agent.invoke({"messages": message_history + [("human", new_query)]})

# Extract the human and AI messages from the result
filtered_messages = [
    msg
    for msg in response["messages"]
    if isinstance(msg, (HumanMessage, AIMessage)) and msg.content.strip()
]

# Pass the new query as input and print the final outputs
print(
    {
        "user_input": new_query,
        "agent_output": [
            f"{msg.__class__.__name__}: {msg.content}" for msg in filtered_messages
        ],
    }
)


## Graphs

### Graph State

Organizes the order of tasks (tool usage and llm calls)

### Agent State

Tracks the agents progress as text

Start and End nodes are prebuilt in langgraph




In [42]:
from typing import Annotated 
from typing_extensions import TypedDict 

from langgraph.graph import StateGraph, START, END 
from langgraph.graph.message import add_messages 

from langchain_openai import ChatOpenAI 

llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model="gpt-4o-mini")

class State(TypedDict): 
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

def chatbot(state:State): 
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [ ]:
def stream_graph_updates(user_input: str):

    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values(): 
            print("Agent:", value["messages"])

user_query = " Who is Charles Bukowski?"

stream_graph_updates(user_query)

In [ ]:
from IPython.display import Image, display 

try:
    display(Image(graph.get_graph().draw_mermaid_png()))

except Exception as e: 
    print(e)

## Adding external tools to a chatbot

In [45]:
from langchain_community.utilities import WikipediaAPIWrapper 
from langchain_community.tools import WikipediaQueryRun 

api_wrapper = WikipediaAPIWrapper(top_k_results=1)

wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

tools = [wikipedia_tool]

llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}



In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[wikipedia_tool])
graph_builder.add_node('tools', tool_node)

graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.add_edge('tools', 'chatbot')
graph_builder.add_edge(START, 'chatbot')
graph_builder.add_edge('chatbot', END)

display(Image(graph_builder.compile().get_graph().draw_mermaid_png()))


In [ ]:
def stream_tool_responses(user_input: str): 
    for event in graph.stream({"messages": [("user", user_input)]}): 
        for value in event.values(): 
            print("Agent:", value["messages"])

user_query = "House of lords"
stream_tool_responses(user_query)   

### Adding memory

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver() 
graph = graph_builder.compile(checkpointer=memory)

def stream_memory_responses(user_input: str):
    
    config={"configurable": {"thread_id": "single_session_memory"}}
    
    for event in graph.stream({"messages": [("user", user_input)]}, config):
        for value in event.values():
            print("Agent:", value["messages"])


user_query = "What is the Colosseum?"
stream_memory_responses(user_query)
user_query = "Who built it?"
stream_memory_responses(user_query)


### Multiple Tools

In [51]:
@tool
def date_checker(date: str) -> str: 
    """Provide a list of important events that happened on a given date in any format."""
    try: 
        answer = llm.invoke(f"List important historical events that occurred on {date}.")
        return answer.content
    except Exception as e: 
        return f"Error retrieving events {str(e)}"  
    
@tool 
def is_palindrome(text: str) -> bool: 
    """Check if a given text is a palindrome."""
    if (text.lower().strip() == text[::-1].lower().strip()):
        return "The text '{text}' is a palindrome."
    else:
        return "The text '{text}' is NOT a palindrome."
    

tools = [wikipedia_tool, date_checker, is_palindrome]




In [ ]:
from langgraph.prebuilt import ToolNode 

tool_node = ToolNode(tools=tools)

model_with_tools = llm.bind_tools(tools)

In [55]:
from langgraph.graph import MessagesState, START, END 

def should_continue(state: MessagesState): 

    last_message = state["messages"][-1]

    if last_message.tool_calls:
        return "tools" 
    
    return END

def call_model(state: MessagesState): 

    last_message = state["messages"][-1] 

    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return {"messages": [AIMessage(content=last_message.total_calls[0]["response"])]}

    return {"messages": [model_with_tools.invoke(state["messages"])]} 


workflow = StateGraph(MessagesState)

workflow.add_node("tools", tool_node)
workflow.add_node("chatbot", call_model)

workflow.add_conditional_edges("chatbot", should_continue, ["tools", END])
workflow.add_edge(START, "chatbot")
workflow.add_edge("tools", "chatbot")
workflow.add_edge("chatbot", END)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

#display(Image(app.get_graph().draw_mermaid_png()))